# Importing Necessary Libraries

In [22]:
import re

def LemNormalize(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Menghapus tanda baca
    return text.split()  # Tokenisasi berdasarkan spasi


In [31]:
import pandas as pd
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import WordNetLemmatizer

# Unduh resource NLTK
nltk.download('wordnet')
nltk.download('stopwords')

# Membaca file CSV secara langsung
data = pd.read_csv('curhatdong_100.csv', encoding='utf-8')

# Preprocessing
lemmer = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    text = text.lower()
    text = ''.join([remove_punct_dict.get(c, c) for c in text])  # Menghapus tanda baca
    return text.split()  # Tokenisasi berdasarkan spasi

# Membuat list pertanyaan dan jawaban
curhat_pertanyaan = data['Pertanyaan'].tolist()
curhat_jawaban = data['Jawaban'].tolist()

# Keyword
GREETING_INPUTS = ("halo", "hi", "hai", "hello", "hei")
GREETING_RESPONSES = [
    "Halo! Cerita apa yang ingin kamu bagikan hari ini?",
    "Hi! Aku di sini untuk mendengarkan.",
    "Halo, apa yang sedang kamu rasakan?",
]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

def build_vsm_responses(user_response):
    """Menggunakan Vector Space Model untuk mencari kecocokan."""
    robo_response = ''

    # Menambahkan input pengguna ke daftar pertanyaan untuk diproses
    curhat_pertanyaan.append(user_response)

    # Membuat representasi vektor TF-IDF
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', token_pattern=None)
    tfidf_matrix = TfidfVec.fit_transform(curhat_pertanyaan)

    # Menghitung kesamaan kosinus
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix)
    similarity_scores = cosine_similarities.flatten()
    sorted_indices = similarity_scores.argsort()[::-1][1:]

    # Mendapatkan hasil terbaik
    if similarity_scores[sorted_indices[0]] == 0:
        robo_response = "Maaf, aku belum punya jawaban untuk itu. Coba ceritakan lebih banyak."
    else:
        best_match_index = sorted_indices[0]
        robo_response = curhat_jawaban[best_match_index]

    # Menghapus input pengguna dari list pertanyaan untuk menjaga konsistensi data
    curhat_pertanyaan.pop()
    return robo_response



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
# Main Chat Loop
print("Halo, Selamat Datang di CurhatDong! Silahkan kamu ingin bercerita apa hari ini?. Ketik 'selesai' untuk keluar.")
while True:
    user_input = input("Anda: ").lower()
    if user_input == 'selesai':
        print("CurhatDong: Terima kasih!")
        break
    elif greeting(user_input):
        print(f"CurhatDong: {greeting(user_input)}")
    else:
        print(f"CurhatDong: {build_vsm_responses(user_input)}")


Halo, Selamat Datang di CurhatDong! Silahkan kamu ingin bercerita apa hari ini?. Ketik 'selesai' untuk keluar.
Anda: aku takut masa depan
CurhatDong: Ketakutan itu wajar. Coba fokus ke langkah kecil yang bisa kamu lakukan hari ini.
Anda: selesai
CurhatDong: Terima kasih!
